# 🔍 RAG 엔진 테스트 (3개 인덱스 + Priority Rerank)

**목표**: 3개 Pinecone 인덱스에서 검색 → Priority Rerank → GPT-4o 답변 생성

**인덱스 구조:**
- `housing-law-index`: 법률 (주택임대차보호법, 민법)
- `housing-rule-index`: 시행령, 시행규칙  
- `housing-case-index`: 판례, 상담사례

**이전 단계**: `embed_to_pinecone_test.ipynb` 완료

## 1️⃣ 라이브러리 임포트 및 API 키 로드

In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv
import textwrap

from langchain_upstage import UpstageEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from pinecone import Pinecone

print("✅ 라이브러리 임포트 완료")

✅ 라이브러리 임포트 완료


In [2]:
# API 키 로드
BASE_DIR = Path.cwd().parents[1]  # 프로젝트 루트
env_path = BASE_DIR / ".env"
load_dotenv(env_path)

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 3개 인덱스명
LAW_INDEX_NAME = "housing-law-index"
RULE_INDEX_NAME = "housing-rule-index"
CASE_INDEX_NAME = "housing-case-index"

# API 키 확인
print("🔑 API 키 확인:")
print(f"  PINECONE: {'✅' if PINECONE_API_KEY else '❌'}")
print(f"  UPSTAGE: {'✅' if UPSTAGE_API_KEY else '❌'}")
print(f"  OPENAI: {'✅' if OPENAI_API_KEY else '❌'}")
print(f"\n📌 인덱스:")
print(f"  - Law: {LAW_INDEX_NAME}")
print(f"  - Rule: {RULE_INDEX_NAME}")
print(f"  - Case: {CASE_INDEX_NAME}")

🔑 API 키 확인:
  PINECONE: ✅
  UPSTAGE: ✅
  OPENAI: ✅

📌 인덱스:
  - Law: housing-law-index
  - Rule: housing-rule-index
  - Case: housing-case-index


## 2️⃣ Upstage 임베딩 & Pinecone 연결

In [3]:
print("📦 Upstage 임베딩 로드 중...")
embedding = UpstageEmbeddings(
    model="solar-embedding-1-large-passage",
    api_key=UPSTAGE_API_KEY
)
print("✅ 임베딩 준비 완료")

📦 Upstage 임베딩 로드 중...
✅ 임베딩 준비 완료


In [4]:
print("🔗 Pinecone 연결 중...")
pc = Pinecone(api_key=PINECONE_API_KEY)

# 3개 VectorStore 초기화
law_vectorstore = PineconeVectorStore(
    index_name=LAW_INDEX_NAME,
    embedding=embedding
)

rule_vectorstore = PineconeVectorStore(
    index_name=RULE_INDEX_NAME,
    embedding=embedding
)

case_vectorstore = PineconeVectorStore(
    index_name=CASE_INDEX_NAME,
    embedding=embedding
)

print("✅ 3개 Pinecone 인덱스 연결 완료")

🔗 Pinecone 연결 중...
✅ 3개 Pinecone 인덱스 연결 완료


## 3️⃣ OpenAI LLM 초기화

In [5]:
print("🤖 OpenAI LLM 초기화 중...")
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.2,
    openai_api_key=OPENAI_API_KEY
)
print("✅ LLM 준비 완료")

🤖 OpenAI LLM 초기화 중...
✅ LLM 준비 완료


## 4️⃣ 카테고리 키워드 사전

In [6]:
def get_law_category():
    """주택임대차 RAG 시스템을 위한 카테고리-키워드 가중치 사전"""
    return {
        '보증금_대항력': {
            '보증금': 3, '대항력': 3, '우선변제권': 3, '최우선변제': 3,
            '보증금반환': 3, '전세금': 2, '소액임차인': 2, '확정일자': 2,
            '전입신고': 2, '점유': 2, '임차권등기': 3, '배당': 2
        },
        '계약갱신': {
            '계약갱신': 3, '갱신요구': 3, '갱신거절': 3, '묵시적갱신': 3,
            '계약연장': 2, '실거주': 3, '존속기간': 2, '재계약': 2
        },
        '계약해지': {
            '계약해지': 3, '해지통고': 3, '중도해지': 3, '기간만료': 2,
            '계약종료': 2, '퇴거': 2, '명도': 2, '합의해지': 2
        },
        '임대료_증감': {
            '차임': 3, '월세': 2, '증액': 3, '감액': 2, '인상': 2,
            '5퍼센트': 3, '5%': 3, '20분의 1': 2, '전환율': 2
        },
        '수선_원상회복': {
            '수선': 3, '수리': 3, '원상회복': 3, '파손': 2, '누수': 2,
            '필요비': 3, '유익비': 3, '비용상환': 2
        },
        '권리_리스크': {
            '전세사기': 3, '깡통전세': 3, '신탁': 3, '근저당': 3, '저당권': 3,
            '선순위': 3, '가압류': 3, '압류': 3, '체납': 3, '독소조항': 2
        },
        '행정절차': {
            '확정일자부여': 3, '동주민센터': 2, '등기소': 2, '임대차계약증서': 2
        },
        '분쟁해결': {
            '분쟁조정': 3, '지급명령': 3, '소송': 3, '경매': 3, '손해배상': 3
        },
        '임차권_승계': {
            '임차권승계': 3, '승계': 3, '사망': 3, '상속': 3, '사실혼': 3
        }
    }

print("✅ 카테고리 사전 정의 완료")

✅ 카테고리 사전 정의 완료


## 5️⃣ 프롬프트 템플릿

In [7]:
template = """당신은 임차인의 권익을 보호하는 전문 법률 상담 AI입니다.

### 중요 규칙:
1. **반드시 근거 법령을 명시**하세요 (예: 주택임대차보호법 제3조)
2. 임차인에게 **불리한 조항은 명확히 경고**하세요
3. **구체적인 해결 방안**을 제시하세요
4. 법률 용어는 쉽게 풀어서 설명하세요

### 검색된 법령 및 사례:
{context}

### 사용자 질문:
{question}

### AI 답변:
"""

prompt_template = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

print("✅ 프롬프트 템플릿 생성 완료")

✅ 프롬프트 템플릿 생성 완료


## 6️⃣ Priority Rerank 함수
# pip install -U cohere

In [8]:
def rerank_by_priority(documents):
    """
    우선순위 기반 rerank
    
    Priority 순서:
    1. 주택임대차보호법 (priority=1)
    2. 시행령 (priority=2)
    3. 시행규칙 (priority=3)
    4. 민법 (priority=4)
    5. 대법원 판례 (priority=5)
    6. 하급심 판례 (priority=6)
    7. 상담 사례 (priority=7)
    """
    reranked = sorted(
        documents,
        key=lambda doc: int(doc.metadata.get('priority', 99))
    )
    return reranked

print("✅ Rerank 함수 정의 완료")

import cohere

def rerank_by_cohere(query, documents, top_k=10):
    """
    Cohere Rerank API로 질문 관련성 기반 재정렬
    
    Args:
        query: 사용자 질문
        documents: 재정렬할 문서 리스트
        top_k: 상위 몇 개 선택할지
    
    Returns:
        재정렬된 문서 리스트
    """
    co = cohere.Client(api_key=COHERE_API_KEY)
    
    # 문서를 텍스트로 변환 (제목 + 내용)
    docs_text = [
        doc.metadata.get('law_name', '') + " " + 
        doc.metadata.get('article', '') + " " + 
        doc.page_content 
        for doc in documents
    ]
    
    # Cohere Rerank API 호출
    results = co.rerank(
        model="rerank-multilingual-v3.0",  # 한국어 지원 모델
        query=query,
        documents=docs_text,
        top_n=top_k
    )
    
    # 재정렬된 순서대로 문서 반환
    reranked = [documents[r.index] for r in results.results]
    return reranked

print("✅ Cohere Rerank 함수 정의 완료")
#```

#**.env 파일에 추가**:
#```
from dotenv import load_dotenv
import os

load_dotenv()
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

✅ Rerank 함수 정의 완료
✅ Cohere Rerank 함수 정의 완료


## 7️⃣ 통합 RAG 질의 함수 (3개 인덱스 검색)

In [9]:
%pip install cohere

Note: you may need to restart the kernel to use updated packages.


In [10]:
def query_rag(question, k_per_index=5, top_n=10):
    """
    3개 인덱스에서 검색 후 통합 Rerank
    
    Args:
        question: 사용자 질문
        k_per_index: 각 인덱스에서 가져올 문서 수
        top_n: 최종 선택할 문서 수
    
    Returns:
        str: AI 답변
    """
    try:
        # 1. 3개 인덱스에서 검색
        print(f"🔍 3개 인덱스 검색 중... (각 {k_per_index}개)")
        
        law_docs = law_vectorstore.similarity_search(question, k=k_per_index)
        print(f"   ✅ Law: {len(law_docs)}개")
        
        rule_docs = rule_vectorstore.similarity_search(question, k=k_per_index)
        print(f"   ✅ Rule: {len(rule_docs)}개")
        
        case_docs = case_vectorstore.similarity_search(question, k=k_per_index)
        print(f"   ✅ Case: {len(case_docs)}개")
        
        # 2. 모든 문서 통합
        all_docs = law_docs + rule_docs + case_docs
        print(f"\n📊 총 검색 결과: {len(all_docs)}개")
        
        # 3. Priority Rerank
        print(f"🔄 Priority Rerank 중...")
        priority_ranked = rerank_by_priority(all_docs)
        
        # 🆕 3.5. Cohere Rerank (여기 추가!)
        print(f"🎯 Cohere Rerank 중...")
        cohere_ranked = rerank_by_cohere(question, priority_ranked[:20], top_n)
        
        # 4. 최종 문서 선택
        top_docs =priority_ranked[:top_n]   # 평가 위해 번갈아 주석처리
        #top_docs = cohere_ranked   
        print(f"   ✅ 상위 {len(top_docs)}개 선택 완료")
        
        # 5. Context 생성
        context_parts = []
        for i, doc in enumerate(top_docs, 1):
            meta = doc.metadata
            priority = int(meta.get('priority', 99))   # ⬅️ 여기만 핵심 수정
            law_name = meta.get('law_name', meta.get('src_title', 'Unknown'))
            article = meta.get('article', '')
            
            content = textwrap.fill(doc.page_content[:300], width=80)
            
            context_parts.append(
                f"[문서 {i}] (우선순위: {priority})\n"
                f"출처: {law_name} {article}\n"
                f"내용: {content}...\n"
            )

        context = "\n".join(context_parts)

        
        # 6. LLM 답변 생성
        print(f"🤖 GPT-4o 답변 생성 중...")
        prompt = prompt_template.format(
            context=context,
            question=question
        )
        
        answer = llm.invoke(prompt).content
        
        #줄바꿈 처리: 80자 이상 문장을 자동 줄바꿈
        lines = answer.split('\n')
        wrapped_lines = []
        for line in lines:
            if len(line) > 80 and not line.strip().startswith('#'):
                # textwrap으로 80자마다 줄바꿈
                wrapped = textwrap.fill(line, width=80, break_long_words=False, break_on_hyphens=False)
                wrapped_lines.append(wrapped)
            else:
                wrapped_lines.append(line)

        answer = '\n'.join(wrapped_lines)

        # 7. 참고 법령 추가
        answer += "\n\n---\n**📚 참고 법령 및 사례:**\n"
        for i, doc in enumerate(top_docs, 1):
            meta = doc.metadata
            law_name = meta.get('law_name', meta.get('src_title', '?'))
            article = meta.get('article', '')
            priority = meta.get('priority', '?')
            
            answer += f"{i}. {law_name} {article} (우선순위: {priority})\n"
        
        answer += "\n⚠️ 본 답변은 AI 분석 결과로, 최종 결정 전 변호사와 상담하시기 바랍니다."
        
        return answer
        
    except Exception as e:
        return f"❌ 오류가 발생했습니다: {str(e)}"

print("✅ 통합 RAG 질의 함수 정의 완료")

✅ 통합 RAG 질의 함수 정의 완료


## 8 통계 확인

In [11]:
# 각 인덱스 통계 확인
print("📊 Pinecone 인덱스 통계:\n")

for idx_name in [LAW_INDEX_NAME, RULE_INDEX_NAME, CASE_INDEX_NAME]:
    try:
        idx = pc.Index(idx_name)
        stats = idx.describe_index_stats()
        print(f"📌 {idx_name}:")
        print(f"   - 총 벡터: {stats['total_vector_count']}")
        print(f"   - 차원: {stats['dimension']}")
        print()
    except Exception as e:
        print(f"❌ {idx_name}: {e}\n")

📊 Pinecone 인덱스 통계:

📌 housing-law-index:
   - 총 벡터: 19
   - 차원: 4096

📌 housing-rule-index:
   - 총 벡터: 4
   - 차원: 4096

📌 housing-case-index:
   - 총 벡터: 160
   - 차원: 4096



## ✅ 완료

**다음 단계:**
1. Django 웹 앱 통합
2. 키워드 기반 인덱스 라우팅 추가
3. Cohere Rerank 도입 (옵션)
4. RAGAS 평가